In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv


In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [3]:
train_data = pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv")
train_data.head()

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,SearchEngine,Zone,Type of Traffic,CustomerType,Gender,Cookies Setting,Education,Marital Status,WeekendPurchase,Made_Purchase
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,...,1.0,9.0,3.0,Returning_Visitor,Not Specified,Deny,Not Specified,Other,0.0,False
1,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,...,2.0,2.0,4.0,Returning_Visitor,Female,Deny,Others,Married,0.0,False
2,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,...,3.0,1.0,4.0,Returning_Visitor,Female,ALL,Others,Married,1.0,False
3,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.4,...,4.0,3.0,3.0,Returning_Visitor,Male,ALL,Diploma,Single,0.0,False
4,1.0,0.0,0.0,0.0,0.0,0.000000,0.20,0.20,0.0,0.0,...,2.0,1.0,5.0,Returning_Visitor,Male,Deny,Diploma,Other,1.0,False


In [4]:
test_data = pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv")
test_data.head()

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,OS,SearchEngine,Zone,Type of Traffic,CustomerType,Gender,Cookies Setting,Education,Marital Status,WeekendPurchase
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,...,1.0,1.0,1.0,1.0,Returning_Visitor,Not Specified,Deny,Not Specified,Single,0.0
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.0,0.0,...,2.0,2.0,1.0,2.0,Returning_Visitor,Not Specified,ALL,Graduate,Married,0.0
2,0.0,0.0,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.0,0.0,...,2.0,2.0,1.0,3.0,Returning_Visitor,Female,Required,Diploma,Other,0.0
3,0.0,0.0,0.0,0.0,2.0,37.000000,0.000000,0.100000,0.0,0.8,...,2.0,2.0,2.0,3.0,Returning_Visitor,Not Specified,Required,Graduate,Other,0.0
4,0.0,0.0,0.0,0.0,16.0,407.750000,0.018750,0.025833,0.0,0.4,...,1.0,1.0,4.0,3.0,Returning_Visitor,Female,Deny,Others,Other,0.0


In [5]:
train_data = train_data.rename(columns={'Cookies Setting':'Cookies_Setting' , 'Type of Traffic': 'Type_of_Traffic','Marital Status': 'MaritalStatus','GoogleMetric:Bounce Rates':'Bounce_Rates','GoogleMetric:Exit Rates':'Exit_Rates','GoogleMetric:Page Values':'Page_Values'})
test_data = test_data.rename(columns={'Cookies Setting':'Cookies_Setting' , 'Type of Traffic': 'Type_of_Traffic','Marital Status': 'MaritalStatus','GoogleMetric:Bounce Rates':'Bounce_Rates','GoogleMetric:Exit Rates':'Exit_Rates','GoogleMetric:Page Values':'Page_Values'})

In [6]:
num_cols = ['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
       'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
       'Bounce_Rates', 'Exit_Rates', 'Page_Values', 'SeasonalPurchase', 'OS',
       'SearchEngine', 'Zone', 'Type_of_Traffic']
cat_cols = ['Month_SeasonalPurchase', 'CustomerType', 'Gender', 'Cookies_Setting',
       'Education', 'MaritalStatus','WeekendPurchase']
print(num_cols)
print(cat_cols)

['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration', 'ProductDescriptionPage', 'ProductDescriptionPage_Duration', 'Bounce_Rates', 'Exit_Rates', 'Page_Values', 'SeasonalPurchase', 'OS', 'SearchEngine', 'Zone', 'Type_of_Traffic']
['Month_SeasonalPurchase', 'CustomerType', 'Gender', 'Cookies_Setting', 'Education', 'MaritalStatus', 'WeekendPurchase']


In [7]:
X_train = train_data.drop('Made_Purchase',axis=1) #features
y_train = train_data['Made_Purchase'] #label
y_train = y_train.astype(int)

In [8]:
print(X_train.describe())

           HomePage  HomePage_Duration   LandingPage  LandingPage_Duration  \
count  14578.000000       14581.000000  14578.000000          14596.000000   
mean       2.250240          79.300762      0.490739             33.455943   
std        3.288042         179.374699      1.252376            140.146256   
min        0.000000           0.000000      0.000000              0.000000   
25%        0.000000           0.000000      0.000000              0.000000   
50%        1.000000           5.000000      0.000000              0.000000   
75%        3.000000          91.000000      0.000000              0.000000   
max       27.000000        3398.750000     24.000000           2549.375000   

       ProductDescriptionPage  ProductDescriptionPage_Duration  Bounce_Rates  \
count            14608.000000                     14564.000000  14580.000000   
mean                31.559488                      1184.346084      0.023366   
std                 44.897089                      2009.4

In [9]:
X_train.isnull().sum()

HomePage                           153
HomePage_Duration                  150
LandingPage                        153
LandingPage_Duration               135
ProductDescriptionPage             123
ProductDescriptionPage_Duration    167
Bounce_Rates                       151
Exit_Rates                         129
Page_Values                        132
SeasonalPurchase                   150
Month_SeasonalPurchase             144
OS                                 134
SearchEngine                       122
Zone                               117
Type_of_Traffic                    143
CustomerType                       144
Gender                             145
Cookies_Setting                    144
Education                          136
MaritalStatus                      130
WeekendPurchase                    121
dtype: int64

In [10]:
print(X_train.Month_SeasonalPurchase.unique())
print(X_train.CustomerType.unique())
print(X_train.Gender.unique())
print(X_train.Cookies_Setting.unique())
print(X_train.Education.unique())
print(X_train.MaritalStatus.unique())
print(X_train.WeekendPurchase.unique())

['Feb' 'Mar' 'May' 'Oct' 'June' 'Jul' 'Aug' 'Nov' 'Sep' 'Dec' nan]
['Returning_Visitor' 'New_Visitor' 'Other' nan]
['Not Specified' 'Female' 'Male' nan]
['Deny' 'ALL' 'Required' nan]
['Not Specified' 'Others' 'Diploma' 'Graduate' nan]
['Other' 'Married' 'Single' nan]
[ 0.  1. nan]


In [11]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14731 entries, 0 to 14730
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HomePage                         14578 non-null  float64
 1   HomePage_Duration                14581 non-null  float64
 2   LandingPage                      14578 non-null  float64
 3   LandingPage_Duration             14596 non-null  float64
 4   ProductDescriptionPage           14608 non-null  float64
 5   ProductDescriptionPage_Duration  14564 non-null  float64
 6   Bounce_Rates                     14580 non-null  float64
 7   Exit_Rates                       14602 non-null  float64
 8   Page_Values                      14599 non-null  float64
 9   SeasonalPurchase                 14581 non-null  float64
 10  Month_SeasonalPurchase           14587 non-null  object 
 11  OS                               14597 non-null  float64
 12  SearchEngine      

In [12]:
# numerical feature
X_num = train_data[num_cols] 
# categorical feature
X_cat = train_data[cat_cols]
# label or target variable
y = train_data['Made_Purchase']

# missing values for numerical features
num_imputer = IterativeImputer(estimator=LinearRegression(),random_state=0)
X_num_imputed = num_imputer.fit_transform(X_num)
# to convert to dataframe
X_num_imputed = pd.DataFrame(X_num_imputed,columns=num_cols)
# missing values for categorical features
cat_imputer = SimpleImputer(strategy='most_frequent')
X_cat_imputed = cat_imputer.fit_transform(X_cat)



In [13]:
scaler = RobustScaler()
batch_size = 14731

for i in range(0,X_num_imputed.shape[0],batch_size):
    X_batch = X_num_imputed[i:i+batch_size]
    scaler.fit(X_batch)
    X_batch_scaled = scaler.transform(X_batch)
X_batch_scaled = pd.DataFrame(X_batch_scaled,columns=num_cols)
print(X_batch_scaled.shape)

(14731, 14)


In [14]:

# converting categorical features to numerical features using one hot encoded
encoder = OneHotEncoder(sparse=False,handle_unknown='ignore')
X_cat_encoded=encoder.fit_transform(X_cat_imputed)
feature_names = encoder.get_feature_names(cat_cols)
X_cat_encoded=pd.DataFrame(X_cat_encoded,columns = feature_names)

# concatenating every column together after preprocessing
X_imputed_encoded = pd.concat([X_batch_scaled, X_cat_encoded], axis=1)
# print(X_imputed_encoded.drop([cat_cols]),axis=1)
data_imputed_encoded = pd.concat([X_imputed_encoded, y], axis=1)

print(data_imputed_encoded.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14731 entries, 0 to 14730
Data columns (total 43 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HomePage                         14731 non-null  float64
 1   HomePage_Duration                14731 non-null  float64
 2   LandingPage                      14731 non-null  float64
 3   LandingPage_Duration             14731 non-null  float64
 4   ProductDescriptionPage           14731 non-null  float64
 5   ProductDescriptionPage_Duration  14731 non-null  float64
 6   Bounce_Rates                     14731 non-null  float64
 7   Exit_Rates                       14731 non-null  float64
 8   Page_Values                      14731 non-null  float64
 9   SeasonalPurchase                 14731 non-null  float64
 10  OS                               14731 non-null  float64
 11  SearchEngine                     14731 non-null  float64
 12  Zone              

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
# numerical feature
X_num_test = test_data[num_cols] 
# categorical feature
X_cat_test = test_data[cat_cols]
# label or target variable
# y = train_data['Made_Purchase']

# missing values for numerical features
X_num_imputed_test = num_imputer.fit_transform(X_num_test)
# to convert to dataframe
X_num_imputed_test = pd.DataFrame(X_num_imputed_test,columns=num_cols)
# missing values for categorical features

X_cat_imputed_test = cat_imputer.fit_transform(X_cat_test)

batch_size = 6599

for i in range(0,X_num_imputed_test.shape[0],batch_size):
    X_batch_test = X_num_imputed_test[i:i+batch_size]
    scaler.fit(X_batch_test)
    X_batch_scaled_test = scaler.transform(X_batch_test)
X_batch_scaled_test = pd.DataFrame(X_batch_scaled_test,columns=num_cols)
print(X_batch_scaled_test.shape)


# converting categorical features to numerical features using one hot encoded

X_cat_encoded_test=encoder.fit_transform(X_cat_imputed_test)

X_cat_encoded_test=pd.DataFrame(X_cat_encoded_test,columns = feature_names)

# concatenating every column together after preprocessing
X_imputed_encoded_test = pd.concat([X_batch_scaled_test, X_cat_encoded_test], axis=1)
# print(X_imputed_encoded.drop([cat_cols]),axis=1

print(X_imputed_encoded_test.info())

(6599, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 42 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HomePage                         6599 non-null   float64
 1   HomePage_Duration                6599 non-null   float64
 2   LandingPage                      6599 non-null   float64
 3   LandingPage_Duration             6599 non-null   float64
 4   ProductDescriptionPage           6599 non-null   float64
 5   ProductDescriptionPage_Duration  6599 non-null   float64
 6   Bounce_Rates                     6599 non-null   float64
 7   Exit_Rates                       6599 non-null   float64
 8   Page_Values                      6599 non-null   float64
 9   SeasonalPurchase                 6599 non-null   float64
 10  OS                               6599 non-null   float64
 11  SearchEngine                     6599 non-null   float64
 12  Zone     

In [16]:
data_imputed_encoded.describe()

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,Bounce_Rates,Exit_Rates,Page_Values,SeasonalPurchase,...,Cookies_Setting_Required,Education_Diploma,Education_Graduate,Education_Not Specified,Education_Others,MaritalStatus_Married,MaritalStatus_Other,MaritalStatus_Single,WeekendPurchase_0.0,WeekendPurchase_1.0
count,14731.000000,14731.000000,14731.000000,14731.000000,14731.000000,14731.000000,14731.000000,14731.000000,14731.000000,14731.000000,...,14731.000000,14731.000000,14731.00000,14731.000000,14731.000000,14731.000000,14731.000000,14731.000000,14731.000000,14731.000000
mean,0.416024,0.816371,0.491972,33.552072,0.451447,0.476856,1.096760,0.515274,4.814161,0.064093,...,0.330392,0.247980,0.25056,0.239291,0.262168,0.321092,0.344987,0.333922,0.767769,0.232231
std,1.091804,1.973204,1.249710,139.863248,1.493259,1.591770,2.755110,1.411588,16.815065,0.201571,...,0.470370,0.431855,0.43335,0.426665,0.439829,0.466912,0.475380,0.471628,0.422270,0.422270
min,-0.379549,-1.115750,-0.031017,-27.207743,-0.649093,-0.631080,-1.295795,-0.747872,-16.163175,-0.004131,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.333333,-0.054885,0.000000,0.000000,-0.366667,-0.324595,-0.193548,-0.334086,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.666667,0.945115,0.000000,0.000000,0.633333,0.675405,0.806452,0.665914,0.000000,0.000000,...,1.000000,0.000000,1.00000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,8.666667,37.253019,24.000000,2549.375000,22.900000,50.316735,10.806452,5.094918,361.763742,1.000000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
X_imputed_encoded_test.describe()

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,Bounce_Rates,Exit_Rates,Page_Values,SeasonalPurchase,...,Cookies_Setting_Required,Education_Diploma,Education_Graduate,Education_Not Specified,Education_Others,MaritalStatus_Married,MaritalStatus_Other,MaritalStatus_Single,WeekendPurchase_0.0,WeekendPurchase_1.0
count,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,...,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000
mean,0.326715,0.788645,0.495462,33.092199,0.459087,0.463560,1.062002,0.490098,4.983215,0.064290,...,0.342931,0.244886,0.247916,0.247310,0.259888,0.327625,0.343537,0.328838,0.774057,0.225943
std,0.850747,1.842763,1.269843,134.417036,1.453027,1.528349,2.665131,1.366951,17.577442,0.202004,...,0.474725,0.430052,0.431836,0.431481,0.438606,0.469383,0.474925,0.469826,0.418234,0.418234
min,-0.334705,-0.284285,-0.082419,-24.483589,-0.861300,-0.455559,-1.104835,-0.751693,-3.800856,-0.025248,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.254970,-0.054795,0.000000,0.000000,-0.333333,-0.317741,-0.175032,-0.338061,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.745030,0.945205,0.000000,0.000000,0.666667,0.682259,0.824968,0.661939,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,6.374259,37.191781,24.000000,2549.375000,22.933333,50.628990,10.851995,5.001544,361.763742,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [18]:
print(data_imputed_encoded.shape)
print(X_imputed_encoded_test.shape)

(14731, 43)
(6599, 42)


In [19]:
feature_col_train = data_imputed_encoded.columns
print(feature_col_train)
feature_col_test = X_imputed_encoded_test.columns
print(feature_col_test)

Index(['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
       'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
       'Bounce_Rates', 'Exit_Rates', 'Page_Values', 'SeasonalPurchase', 'OS',
       'SearchEngine', 'Zone', 'Type_of_Traffic', 'Month_SeasonalPurchase_Aug',
       'Month_SeasonalPurchase_Dec', 'Month_SeasonalPurchase_Feb',
       'Month_SeasonalPurchase_Jul', 'Month_SeasonalPurchase_June',
       'Month_SeasonalPurchase_Mar', 'Month_SeasonalPurchase_May',
       'Month_SeasonalPurchase_Nov', 'Month_SeasonalPurchase_Oct',
       'Month_SeasonalPurchase_Sep', 'CustomerType_New_Visitor',
       'CustomerType_Other', 'CustomerType_Returning_Visitor', 'Gender_Female',
       'Gender_Male', 'Gender_Not Specified', 'Cookies_Setting_ALL',
       'Cookies_Setting_Deny', 'Cookies_Setting_Required', 'Education_Diploma',
       'Education_Graduate', 'Education_Not Specified', 'Education_Others',
       'MaritalStatus_Married', 'MaritalStatus_Other', '

In [20]:
Xtrain = data_imputed_encoded
Xtest = X_imputed_encoded_test

In [21]:

Xtrain = Xtrain.drop('Made_Purchase',axis=1)
print(Xtrain.shape)
print(Xtrain.columns)

(14731, 42)
Index(['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
       'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
       'Bounce_Rates', 'Exit_Rates', 'Page_Values', 'SeasonalPurchase', 'OS',
       'SearchEngine', 'Zone', 'Type_of_Traffic', 'Month_SeasonalPurchase_Aug',
       'Month_SeasonalPurchase_Dec', 'Month_SeasonalPurchase_Feb',
       'Month_SeasonalPurchase_Jul', 'Month_SeasonalPurchase_June',
       'Month_SeasonalPurchase_Mar', 'Month_SeasonalPurchase_May',
       'Month_SeasonalPurchase_Nov', 'Month_SeasonalPurchase_Oct',
       'Month_SeasonalPurchase_Sep', 'CustomerType_New_Visitor',
       'CustomerType_Other', 'CustomerType_Returning_Visitor', 'Gender_Female',
       'Gender_Male', 'Gender_Not Specified', 'Cookies_Setting_ALL',
       'Cookies_Setting_Deny', 'Cookies_Setting_Required', 'Education_Diploma',
       'Education_Graduate', 'Education_Not Specified', 'Education_Others',
       'MaritalStatus_Married', 'MaritalStat

In [22]:
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import  GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score

In [23]:


gb = GradientBoostingClassifier(random_state=42)
params = {
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'n_estimators': [100, 500]
}

grid_search = GridSearchCV(gb, params, cv=5, scoring='f1', n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(Xtrain,y_train)
# Print the best hyperparameters found by GridSearchCV
print('Best hyperparameters: ', grid_search.best_params_)



Best hyperparameters:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}


In [24]:
from matplotlib import pyplot as plt
grid_search.score(Xtrain,y_train)
pred = grid_search.predict(Xtrain)
print(classification_report(y_train,pred))


test_pred = grid_search.predict(Xtest)
test_pred = test_pred.astype(bool)

              precision    recall  f1-score   support

           0       0.68      0.93      0.78      9065
           1       0.72      0.28      0.41      5666

    accuracy                           0.68     14731
   macro avg       0.70      0.61      0.59     14731
weighted avg       0.69      0.68      0.64     14731



In [25]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import ConfusionMatrixDisplay, classification_report
# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# rf.fit(Xtrain,y_train)


# rf.score(Xtrain,y_train)
# pred = rf.predict(Xtrain)
# print(classification_report(y_train,pred))

# # ConfusionMatrixDisplay.from_predictions(y_train,pred)
# test_pred = rf.predict(Xtest)
# test_pred = test_pred.astype(bool)

In [26]:
test_pred

array([False, False, False, ..., False, False, False])

In [27]:
submission=pd.DataFrame(columns=['id', 'Made_Purchase'])
submission['id']=[i for i in range(len(test_pred))]
submission["Made_Purchase"]=[i for i in test_pred]
submission.to_csv('submission.csv',index=False)
submission



,id,Made_Purchase
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False
...,...,...
6594,6594,False
6595,6595,False
6596,6596,False
6597,6597,False
